In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)


In [ ]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_id = test['PassengerId']
# train_profile = ProfileReport(data,title='train')
# train_profile.to_notebook_iframe()
# train_profile.to_file('train_profile.html')
# test_profile = ProfileReport(test,title='test')
# test_profile.to_notebook_iframe()
# test_profile.to_file('test_profile.html')


In [ ]:
data.head(10)

In [ ]:
data.drop(['Name'],axis=1,inplace=True)
test.drop(['Name'],axis=1,inplace=True)

In [ ]:
sns.heatmap(data.isnull(),yticklabels=False,cbar=False)

In [ ]:
data.isnull().sum()
data.dtypes
test.isnull().sum()

In [ ]:
numeric = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
object_data = ['HomePlanet','CryoSleep','Cabin','Destination','VIP']


In [ ]:
# def medianNumFill(datas):
#     for num in datas:
#         datas[num] = datas[num].fillna(datas[num].median())
#     return datas

In [ ]:
# medianNumFill(numeric)
# medianNumFill(numeric_test)
# data['Age'].plot(kind="hist",title="Count of Ages")
# # def clean_objects(dataset_objects):
# #     for object in dataset_objects:
# #         dataset_objects[object] = dataset_objects[object].fillna(dataset_objects[object].mode()[0])
# #     return dataset_objects
# clean_objects(object_test)
# clean_objects(object_data)
moded = data[object_data].mode().iloc[0]
data[object_data] = data[object_data].fillna(moded)
modet = test[object_data].mode().iloc[0]
test[object_data] = test[object_data].fillna(modet)
medd = data[numeric].median()
data[numeric] = data[numeric].fillna(medd)
medt = test[numeric].median()
test[numeric] = test[numeric].fillna(medt)


In [ ]:
data['Deck'] = data['Cabin'].str[0]
data['Side'] = data['Cabin'].str[-1]
test['Deck'] = test['Cabin'].str[0]
test['Side'] = test['Cabin'].str[-1]
data.drop(['Cabin'],axis=1,inplace=True)
test.drop(['Cabin'],axis=1,inplace=True)


In [ ]:
data['Deck'].value_counts()

In [ ]:
sns.heatmap(data.isnull(),yticklabels=False,cbar=False)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cols = ['HomePlanet','CryoSleep','Destination','VIP','Deck','Side']

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)
data['Transported'] = le.fit_transform(data['Transported'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 

In [ ]:
y = data['Transported']
x= data.drop(['Transported'],axis=1)
X_train,X_val,Y_train,Y_val = train_test_split(x,y,test_size=0.15,random_state=42)

In [ ]:
regressor = RandomForestRegressor(random_state=0,).fit(X_train,Y_train)

In [ ]:
predictions = regressor.predict(X_val)
threshold = 0.50
predictions = np.where(predictions >= threshold, 1, 0)

from sklearn.metrics import accuracy_score
accuracy_score(Y_val,predictions)



In [ ]:
sub_pred = regressor.predict(test)

In [ ]:
sub_pred = np.where(sub_pred >= threshold, True, False)

In [ ]:
df = pd.DataFrame({"PassengerId":test_id.values,
                   "Transported":sub_pred
                   })

In [ ]:
df.to_csv("submission.csv",index=False)